# Parallelization

Even for large cases _foxes_ calculations are fast, thanks to

- **Vectorization:** The states (and also the points, in the case of data calculation at evaluation points) are split into so-called _chunks_, which are sub-arrays of the large original data.
- **Parallelization:** These chunks are being sent to individual processes for calculation. Those calculations can be carried out simultaneously, i.e., _in parallel_.  

Vectorization and parallelization are managed by so-called _engines_ in _foxes_. If you do not explicitly specify the engine, a default will be chosen. This means that even if you do not know or care about _foxes_ engines, your calculations will be vectorized and parallelized.

## Available engines

These are the currently available engines, where each can be addressed by the short name or the full class name:

| Short name    | Class name         | Base package | Description                        |
|---------------|--------------------|--------------|------------------------------------| 
| threads       | ThreadsEngine      | [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs on a workstation/laptop,  sends chunks to threads |
| process       | ProcessEngine      | [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs on a workstation/laptop,  sends chunks to parallel processes |
| multiprocess  | MultiprocessEngine | [multiprocess](https://github.com/uqfoundation/multiprocess) |  Runs on a workstation/laptop,  sends chunks to parallel processes |
| ray  | RayEngine | [ray](hhttps://docs.ray.io/en/latest/) |  Runs on a workstation/laptop,  sends chunks to parallel processes |
| dask          | DaskEngine         | [dask](https://www.dask.org/) | Runs on a workstation/laptop, using processes or threads |
| xarray        | XArrayEngine       | [xarray](https://docs.xarray.dev/en/stable/) | Runs on a workstation/laptop, involving [dask](https://www.dask.org/) through [apply_ufunc](https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html)|
| local_cluster | LocalClusterEngine | [distributed](https://distributed.dask.org/en/stable/) | Runs on a workstation/laptop, creates a virtual local cluster |
| slurm_cluster | SlurmClusterEngine | [dask_jobqueue](https://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html) | Runs on a multi-node HPC cluster which is using SLURM |
| mpi           | MPIEngine          | [mpi4py](https://mpi4py.readthedocs.io/en/stable/index.html) | Runs on laptop/workstation/cluster, also supports multi-node runs |
| numpy         | NumpyEngine        | [numpy](https://numpy.org/) | Runs a loop over chunks, without parallelization |
| single        | SingleChunkEngine  | [numpy](https://numpy.org/) | Runs all in a single chunk, without parallelization |
| default       | DefaultEngine      | [numpy](https://numpy.org/), [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) | Runs either the `single` or the `process` engine, depending on the case size |

Note that the external packages are not installed by default. You can install them manually on demand, or use the option `pip install foxes[eng]` for the complete installation of all requirements of the complete list of engines.

Furthermore, scripts that use the `mpi` engine have to be started in a special way. For example, when running a script named `run.py` on 12 processors, the terminal command is
```console
mpiexec -n 12 python -m mpi4py.futures run.py
```

## Default engine

Let's start by importing _foxes_ and other required packages:

In [ ]:
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV

Next, we create a random wind farm and a random time series:

In [ ]:
n_times = 5000
n_turbines = 100
seed = 42

sdata = foxes.input.states.create.random_timseries_data(
    n_times,
    seed=seed,
)
states = foxes.input.states.Timeseries(
    data_source=sdata,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.02},
)

farm = foxes.WindFarm()
foxes.input.farm_layout.add_random(
    farm, n_turbines, min_dist=500, turbine_models=["DTU10MW"], seed=seed, verbosity=0
)

In [ ]:
sdata

In [ ]:
foxes.output.FarmLayoutOutput(farm).get_figure(figsize=(6, 6))
plt.show()

You can run the wind farm calculations by simply creating an algorithm and calling _farm\_calc_:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=1,
)

In [ ]:
farm_results = algo.calc_farm()
farm_results

During the very first calculation, the algorithm checks if an engine is already up and running. If not, the default engine is created. We can check the currently active engine by the following function:

In [ ]:
foxes.get_engine()

This shows that the default choice is the _DefaultEngine_, which selected to run the _ProcessEngine_ for this size of problem. The criteria are:

- If `n_states >= sqrt(n_procs) * (500/n_turbines)**1.5`: Run engine `ProcessEngine`,
- Else if `algo.calc_points()` has been called and `n_states*n_points > 10000`: Run engine `ProcessEngine`,
- Else: Run engine `SingleChunkEngine`.

The above selection is based on test runs on a Ubuntu workstation with 64 physical cores and might not be the optimal choice for your system. Be aware of this whenever relying on the default engine for smallish cases - if in doubt, better explicitly specify the engine.

Note that the parameter choice _None_ for the chunk sizes represents a default choice by the engine, and does not mean that there is no chunking in the corresponding dimension.

We can reset the engine by

In [ ]:
foxes.reset_engine()

such that no engine is active afterwards:

In [ ]:
print(foxes.get_engine(error=False, default=False))

## Engine selection through the algorithm

There are two ways how to select a non-default engine and set all its parameters, as we will explore in this and the following section. 

If you are using one algorithm for all calculations, you can select the engine directly via the algorithm's constructor. Make sure the algorithm is created at the beginning of your script, in particular before creating images, since those might launch the default engine otherwise.

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    engine="dask",
    n_procs=2,
    chunk_size_states=2000,
    chunk_size_points=4000,
    verbosity=1,
)

Here the _DaskEngine_ class was selected, with _n\_procs_ value 2 and a user choice of chunk sizes. Notice that the short name from the above table can be used instead of the full class name (which, however, would also work).

For the complete list of constructor arguments of each of the engine classes, please check the API section _foxes.engines_. Any argument of the engine constructor can directly be added to the constructor of the algorithm, and will then be passed on.

Let's re-run the calculation using the above selected engine:

In [ ]:
farm_results = algo.calc_farm()

We can always check the current engine, and reset it if desired:

In [ ]:
foxes.get_engine()

In [ ]:
foxes.reset_engine()
print(foxes.get_engine(error=False, default=None))

For a proper shutdown of the applied engine, make sure you do not forget the `foxes.reset_engine()` command after the final calculation. Alternatively, consider the selection of the engine by a `with` block, as explained in the following section. 

## Engine selection through a with-block

The proper way of using an engine is recommended to apply a Python context manager, i.e., a _with_ block, for creating the engine. This ensures the proper shutdown of the engine, and it also increases the readability concerning the engine choice.

Especially for cluster and pool based engines, which have non-trivial shutdown routines, the _with_ block is always preferrable over the algorithm based engine specification.

The syntax is straight forward. Note that within the context block we create the algorithm without any engine specification. Furthermore, note that the engine object is not required as a parameter for the algorithm, since it is set as a globally accessible object during initialization:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=0,
)

with foxes.Engine.new(
    "local_cluster", n_procs=4, chunk_size_states=2000, chunk_size_points=10000
):
    farm_results = algo.calc_farm()

    o = foxes.output.FlowPlots2D(algo, farm_results)
    g = o.gen_states_fig_xy(FV.WS, resolution=30, figsize=(6, 6), states_isel=[0])
    next(g)
    plt.show()

Notice the _Dashboard_ link which for this particular choice of engine displays the progress and cluster load during the execution. 

After the computation the engine is not set, as expected:

In [ ]:
print(foxes.get_engine(error=False, default=False))

## Manual engine selection

It can be useful to create the `Engine` object manually, especially when working in a notebook. In that case the engine is often needed many times, and switching it on and off again all the time is not very efficient. 

For such cases, create the engine in the beginning and initialize it:

In [ ]:
engine = foxes.Engine.new("process", n_procs=4, chunk_size_states=2000)
engine.initialize()

Afterwards, run all of your code with that engine:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=0,
)

farm_results = algo.calc_farm()
print(farm_results[FV.REWS])

After everything is done, shutdown the engine:

In [ ]:
engine.finalize()

The `engine` object still exists after this, so you can always decide to initialize/finalize it again, if needed.

## Remarks & recommendations

- Take the time to think about your engine choice, and its parameters. Your choice might matter a lot for the performance of your run.
- In general, all engines accept the parameters `n_procs`, `chunk_size_states`, `chunk_size_points` (the `single` engine ignores them, though).
- If `n_procs` is not set, the maximal number of processes is applied, according to `os.cpu_count()` for Python version < 3.13 and `os.process_cpu_count()` for Python version >= 3.13.
- If `chunk_size_states` is not set, the number of states is divided by `n_procs`. This might be non-optimal for small cases.
- If `chunk_size_points` is not set and there is more than one states chunk, the full number of points is selected such that there is only one point chunk for each state chunk. If there is only one states chunk, the default points chunk size is the number of points divided by `n_procs`.
- In general, for not too small cases, the default `process` engine is a good choice for runs on a linux based laptop or a workstation computer, or within Windows WSL.
- For runs on native Windows, i.e., without WSL, the best engine choices have not been tested. Make sure you try different ones, e.g. `process`, `multiprocess`, `dask`, `numpy`, and also vary the parameters.
- The `mpi` engine requires the installation of MPI on the system, for example _OpenMPI_.
- If you run into memory problems, the best options are to either reduce the number of processes or the chunk sizes.
- The `dask` engine has additional options, accessible through the _dask\_pars_ dictionary parameter, for example the _scheduler_ choice. See API and [dask documentation](https://docs.dask.org/en/stable/scheduling.html) for syntax and more info.
- The `local_cluster` is not always faster than the `process`, `multiprocess` or `dask` engines, but offers a more detailed setup. For example, the memory and the number of threads per worker can be modified, if needed.
- The `numpy` and `single` engines are intended for testing and small cases, and also for sequential runs without large point evaluations, or for smallish runs with wake frame `dyn_wakes`.
- In notebooks, the preferred engine selection method is the `Manual engine selection` described above.